In [1]:
!pip install transformers datasets evaluate tensorflow 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import collections
import numpy as np
from tqdm.auto import tqdm
import pandas as pd
import datasets
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering, DefaultDataCollator, create_optimizer
import tensorflow as tf
import evaluate
import json

In [3]:
squad = load_dataset('esakrissa/squad_id_v2')

  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
pd_squad_train = pd.DataFrame(squad['train'])
pd_squad_val = pd.DataFrame(squad['validation'])

In [5]:
pd_squad_train = pd_squad_train[pd_squad_train.answers != {'text': [], 'answer_start': []}]
pd_squad_val = pd_squad_val[pd_squad_val.answers != {'text': [], 'answer_start': []}]

In [6]:
dataset_train = Dataset.from_dict(pd_squad_train)

In [7]:
dataset_val = Dataset.from_dict(pd_squad_val)

In [8]:
dataset_clean = datasets.DatasetDict({"train":dataset_train,"validation":dataset_val})

squad = dataset_clean

In [9]:
tokenizer = AutoTokenizer.from_pretrained("esakrissa/IndoBERT-SQuAD")

In [10]:
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [11]:
train_dataset = squad["train"].map(preprocess_training_examples, batched=True, remove_columns=squad["train"].column_names)

  0%|          | 0/87 [00:00<?, ?ba/s]

In [12]:
train_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 87389
})

In [13]:
def preprocess_validation_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [14]:
validation_dataset = squad['validation'].map(preprocess_validation_function, batched=True, remove_columns=squad["validation"].column_names)

  0%|          | 0/6 [00:00<?, ?ba/s]

In [15]:
small_eval_set = squad["validation"].select(range(100))
eval_set = small_eval_set.map(preprocess_validation_function, batched=True, remove_columns=squad["validation"].column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [16]:
validation_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 6032
})

In [17]:
eval_set_for_model = eval_set.remove_columns(["example_id", "offset_mapping"])
eval_set_for_model.set_format("numpy")

batch = {k: eval_set_for_model[k] for k in eval_set_for_model.column_names}
trained_model = TFAutoModelForQuestionAnswering.from_pretrained("esakrissa/IndoBERT-SQuAD", from_pt=True)

outputs = trained_model(**batch)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForQuestionAnswering: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


In [18]:
start_logits = outputs.start_logits.numpy()
end_logits = outputs.end_logits.numpy()

In [19]:
def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -20 - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -20 - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > 30
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [20]:
metric = evaluate.load("squad")

In [21]:
data_collator = DefaultDataCollator(return_tensors="tf")

In [22]:
model = TFAutoModelForQuestionAnswering.from_pretrained("esakrissa/IndoBERT-SQuAD", from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForQuestionAnswering: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


In [23]:
tf_train_dataset = model.prepare_tf_dataset(train_dataset, collate_fn=data_collator, shuffle=True, batch_size=4)
tf_eval_dataset = model.prepare_tf_dataset(validation_dataset, collate_fn=data_collator, shuffle=False, batch_size=4)

In [24]:
tf_train_dataset

<PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(4, 384), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(4, 384), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(4, 384), dtype=tf.int64, name=None)}, {'start_positions': TensorSpec(shape=(4,), dtype=tf.int64, name=None), 'end_positions': TensorSpec(shape=(4,), dtype=tf.int64, name=None)})>

In [25]:
num_train_epochs = 3
num_train_steps = len(tf_train_dataset) * num_train_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-05,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer, metrics=['accuracy'])

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
history = model.fit(tf_train_dataset, epochs=num_train_epochs, validation_data=tf_eval_dataset,  validation_steps=1)

Epoch 1/3
   91/21847 [..............................] - ETA: 2:20:06 - loss: 1.0438 - end_logits_accuracy: 0.7170 - start_logits_accuracy: 0.7088

In [ ]:
model.evaluate(tf_train_dataset)

In [ ]:
predictions = model.predict(tf_eval_dataset)
history2 = compute_metrics(
    predictions["start_logits"],
    predictions["end_logits"],
    validation_dataset,
    squad["validation"],
)



In [ ]:
model.save('/content/model/model.tf')

In [ ]:
with open('/content/model/history.json','w') as h:
  json.dump(history.history, h)

with open('/content/model/history.json', 'r') as h:
    history = json.load(h)


In [ ]:
history